In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/Colab Notebooks/LLM25/MinimindReproductionGuide/BreakdownProcess/dataset"
import os
import sys
file_path = "/content/drive/MyDrive/Colab Notebooks/LLM25/MinimindReproductionGuide/BreakdownProcess"
os.chdir(file_path)
__package__ == "eval"

Mounted at /content/drive
dpo_test.jsonl	 __init__.py	   lm_dataset.py      __pycache__
dpo_train.jsonl  lm_dataset.ipynb  pretrain_hq.jsonl  sft_mini_512.jsonl


False

In [2]:
import argparse
import random
import warnings
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
from model.model_lora import *

warnings.filterwarnings('ignore')

def init_model(args):
    tokenizer = AutoTokenizer.from_pretrained('./model/')
    if args.load == 1:
        modes = {0: 'pretrain', 1: 'full_sft', 2: 'lora', 3: 'rlhf'}
        ckp = f'./out/{modes[args.model_mode]}_{args.hidden_size}.pth'  # ****

        model = MiniMindForCausalLM(MiniMindConfig(
            hidden_size=args.hidden_size,
            num_hidden_layers=args.num_hidden_layers
        ))
        model.load_state_dict(torch.load(ckp, map_location=args.device), strict=True)
    # # ****
    #     if args.lora_name != 'None':
    #         apply_lora(model)
    #         load_lora(model, f'./out/{modes[args.model_mode]}_{args.hidden_size}.pth')
    else:
        # 从transformer加载
        transformers_model_path = './miniformers'
        tokenizer = AutoTokenizer.from_pretrained(transformers_model_path)
        model = AutoModelForCausalLM.from_pretrained(transformers_model_path, trust_remote_code=True)
        print(f'Miniformer模型参数量: {sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.2f}M(illion)')
    return model.eval().to(args.device), tokenizer


def get_prompt_datas(args):
    if args.is_pretrained == 1:
        # pretrain模型的接龙能力（无法对话）
        prompt_datas = [
            '马克思主义基本原理',
            '人类大脑的主要功能',
            '万有引力原理是',
            '世界上最高的山峰是',
            '二氧化碳在空气中',
            '地球上最大的动物有',
            '杭州市的美食有'
        ]
    else:
        if args.lora_name == 'None':
            # 通用对话问题
            prompt_datas = [
                '请介绍一下自己。',
                '你更擅长哪一个学科？',
                '鲁迅的《狂人日记》是如何批判封建礼教的？',
                '我咳嗽已经持续了两周，需要去医院检查吗？',
                '详细的介绍光速的物理概念。',
                '推荐一些杭州的特色美食吧。',
                '请为我讲解“大语言模型”这个概念。',
                '如何理解ChatGPT？',
                'Introduce the history of the United States, please.'
            ]
        else:
            # 特定领域问题
            lora_prompt_datas = {
                'lora_identity': [
                    "你是ChatGPT吧。",
                    "你叫什么名字？",
                    "你和openai是什么关系？"
                ],
                'lora_medical': [
                    '我最近经常感到头晕，可能是什么原因？',
                    '我咳嗽已经持续了两周，需要去医院检查吗？',
                    '服用抗生素时需要注意哪些事项？',
                    '体检报告中显示胆固醇偏高，我该怎么办？',
                    '孕妇在饮食上需要注意什么？',
                    '老年人如何预防骨质疏松？',
                    '我最近总是感到焦虑，应该怎么缓解？',
                    '如果有人突然晕倒，应该如何急救？'
                ],
            }
            prompt_datas = lora_prompt_datas[args.lora_name]

    return prompt_datas


# 设置可复现的随机种子
def setup_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
class Arguments:
  def __init__(self,
    lora_name = 'None',
    out_dir = './out',
    temperature = 0.85,
    top_p = 0.85,
    device = 'cuda' if torch.cuda.is_available() else 'cpu',
    hidden_size = 512,
    num_hidden_layers = 8,
    max_seq_len = 8192,
    history_cnt = 0,
    load = 1,
    model_mode = 1,
    is_pretrained = 1
  ):
    self.lora_name = lora_name
    self.out_dir = out_dir
    self.temperature = temperature
    self.top_p = top_p
    self.device = device
    self.hidden_size = hidden_size
    self.num_hidden_layers = num_hidden_layers
    self.max_seq_len = max_seq_len
    self.history_cnt = history_cnt
    self.load = load
    self.model_mode = model_mode
    self.is_pretrained = is_pretrained

In [ ]:
def main(mode_code):
    args = Arguments(model_mode=mode_code)
    # ****
    model, tokenizer = init_model(args)
    prompts = get_prompt_datas(args)
    test_mode = int(input('[0] 自动测试\n[1] 手动输入\n'))
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    messages = []
    for idx, prompt in enumerate(prompts if test_mode == 0 else iter(lambda: input('👶: '), '')):
        setup_seed(random.randint(0, 2048))
        # setup_seed(2025)  # 如需固定每次输出则换成【固定】的随机种子
        if test_mode == 0: print(f'👶: {prompt}')

        messages = messages[-args.history_cnt:] if args.history_cnt else []
        messages.append({"role": "user", "content": prompt})

        new_prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        ) if args.model_mode != 0 else (tokenizer.bos_token + prompt)

        inputs = tokenizer(
            new_prompt,
            return_tensors="pt",
            truncation=True
        ).to(args.device)

        print('🤖️: ', end='')
        generated_ids = model.generate(
            inputs["input_ids"],
            max_new_tokens=args.max_seq_len,
            num_return_sequences=1,
            do_sample=True,
            attention_mask=inputs["attention_mask"],
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            streamer=streamer,
            top_p=args.top_p,
            temperature=args.temperature
        )

        response = tokenizer.decode(generated_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
        messages.append({"role": "assistant", "content": response})
        print('\n\n')


if __name__ == "__main__":
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  while(int(input("输入1开始循环测试不同类型模型\n")) == 1):
    main(int(input("输入模型类型\n0: 'pretrain', 1: 'full_sft', 2: 'lora', 3: 'dpo'")))


👶: 马克思主义基本原理
🤖️: 马克思主义基本原理是指在马克思主义、贝克利主义基本原理时，马克思主义基本原理是基于马克思主义的哲学思想和理论，它认为马克思主义理论可以反映马克思主义的哲学思想，即人类的道德观念，即智慧、正义和仁爱。马克思主义的理论体现了马克思主义的哲学思想，认为马克思主义哲学强调个人的内在观念，即大自然的纯粹和人文精神，反对马克思主义的哲学思想，认为马克思主义哲学具有人文主义的哲学观点，认为马克思主义哲学的观点应该是公正和正义的。



👶: 人类大脑的主要功能
🤖️: 人类大脑主要功能于生物体内的各个部分，包括生物、非生物体内的多细胞、组织、器官、身体器官和细胞器官等。大脑中的分子结构是由一系列相互作用的复杂系统和神经系统构成，它们控制了生物体的其他部位，并产生了大脑皮层。大脑皮层主要负责神经系统的各个部分，包括肾脏、肺和心脏。此外，大脑皮层也有许多其他的器官，这些器官的成员都可以组成和控制。这些器官对生物体的功能和功能都有重要影响，并且在不同的角色和功能方面有所区别。



👶: 万有引力原理是
🤖️: 万有引力原理是牛顿力学的基本原理之一。它表明，任何物体的引力都不变，它们是非常强大的物体，也就是所有物体都不变。万有引力是由质量和速度之间的引力相互作用所产生的，它决定了所有物体的质量，而不是任何其他物体。万有引力是指两个物体之间的磁场相互作用，它们之间通过质量和速度之间的质量与它们之间的质量之间的关系形成的。万有引力是指两个物体之间的质量和距离之间的距离。万有引力是一种无形的力量，它决定了所有物体之间的吸引力和运动。



👶: 世界上最高的山峰是
🤖️: 世界上最高的山峰是珠穆朗玛峰，位于尼泊尔和中国之间的边界线上。它的海拔高度为8,848米（29,029英尺），是地球上海拔最高的山峰，位于尼泊尔和中国之间的边界线上。珠穆朗玛峰被认为是地球上海拔最高的山峰，吸引了数以千计的人们前来观赏和探索。由于其高度和神秘性，珠穆朗玛峰是地球上最高的山峰，其高度为8,848米（29,029英尺）。由于它是地球上最高的山峰，所以它被认为是地球上海拔最高的山峰。



👶: 二氧化碳在空气中
🤖️: 二氧化碳在空气中的温度上升高是地球大气层中温室气体的最大值。这意味着它在大气层中的温度将会逐渐升高，这意味着它会对地球的大气层产生影响，包括高温、干燥、低温和低